In [87]:
import numpy as np

In [88]:
class SFR:
    def __init__(self, stage:int, init_seq, control) -> None:
        self.stage = stage
        self.container = np.asarray(init_seq)
        if type(control) == list:
            self.c_seq = control
            self.func = self.func_specific
        elif control == 1:
            self.func = self.func_1
        elif control == 2:
            self.func = self.func_2
    def func_specific(self):
        res = 0
        for i in range(self.stage):
            res ^= self.container[i] * self.c_seq[i]
        return res

    def func_1(self):
        return self.container[0] ^ self.container[3] ^ (self.container[1] & self.container[2])
    def func_2(self):
        return self.container[0] ^ self.container[2]

    def step_n(self, step:int):
        for _ in range(step):
            output = self.func()
            self.container = np.concatenate((output, self.container[:-1]), axis=None)
    def output_tail(self):
        return self.container[-1]
    def output_head(self):
        return self.container[0]
    def print_status(self):
        print(self.container)
    def print_full_cycle(self):
        output_seq = []
        for _ in range(2**(self.stage+1)):
            output_seq.append(self.output_tail())
            self.step_n(1)
        print(output_seq)
    def get_full_cycle(self):
        output_seq = []
        for _ in range(2**(self.stage+1)):
            output_seq.append(self.output_tail())
            self.step_n(1)
        return output_seq

In [89]:
def dex_to_bin(x:int, *,width=5):
    res = []
    while x != 0:
        res.append(x%2)
        x//=2
    while len(res) < width:
        res.append(0)
    return res[::-1]

### Question 1.

In [90]:
lsfr_1 = SFR(4, np.array([1,1,0,1]), 1)
lsfr_1.print_full_cycle()


[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]


- 周期为5
- 周期序列为 1,0,1,1,0

### Question 2.

In [91]:
lsfr_2 = SFR(3, np.array([0,0,1]), 2)
lsfr_2.print_full_cycle()

[1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0]


In [92]:
lsfr_2 = SFR(3, np.array([1,1,0]), 2)
lsfr_2.print_full_cycle()

[0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1]


可以发现只是位移了两位

### Question 3.

In [93]:
cipher = 0b1011110000110001001010110001
plain_text = 0b1001001001101101100100100110
key = dex_to_bin(cipher ^ plain_text, width=28)
print(key)

[0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1]


(1) 周期为7，可以认为阶数至少为3

(2) 初始状态为100

In [94]:
lsfr_3 = SFR(3, [1,0,0], [0,1,1])
lsfr_3.print_full_cycle()

[0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0]


(3) 穷举了一下，经测试C = [0,1,1]是正确的反馈函数参数 

即 $f(\vec a) = a_{2} \oplus a_{3}$

### Question 4.

In [95]:
alpha = [chr(i+65) for i in range(26)] + [str(j) for j in range(6)]
print(alpha[0b11001])
dictionary = {}
for i in range(32):
    dictionary[alpha[i]] = i

Z


阶数$m = 6$，最大周期为$2^6-1 = 63$

前三个字母的长度为15，WPI --> J5A

In [103]:

first_3_cipher = dex_to_bin(dictionary['J']) + dex_to_bin(dictionary['5']) + dex_to_bin(dictionary['A']) 
print(first_3_cipher)
first_3_plain = dex_to_bin(dictionary['W']) + dex_to_bin(dictionary['P']) + dex_to_bin(dictionary['I']) 
print(first_3_plain)

first_15_output =list( np.asarray(first_3_plain) ^ np.asarray(first_3_cipher))
print(first_15_output)

[0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]


初始状态为111111

In [120]:
for i in range(2**6):
    lsfr_3 = SFR(6, [1,1,1,1,1,1], dex_to_bin(i, width=6))
    if first_15_output[:] == lsfr_3.get_full_cycle()[:15]:
        print(i)
print(dex_to_bin(3, width=6))
lsfr_3 = SFR(6, [1,1,1,1,1,1], dex_to_bin(3, width=6))
lsfr_3.print_full_cycle()
lsfr_3 = SFR(6, [1,1,1,1,1,1], dex_to_bin(3, width=6))

3
[0, 0, 0, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1]


穷举发现只有[0,0,0,0,1,1]这组函数符合

即$f(\vec a) = a_5\oplus a_6$

In [116]:
cipher_bin = \
    dex_to_bin(dictionary['J']) + \
    dex_to_bin(dictionary['5']) + \
    dex_to_bin(dictionary['A']) + \
    dex_to_bin(dictionary['0']) + \
    dex_to_bin(dictionary['E']) + \
    dex_to_bin(dictionary['D']) + \
    dex_to_bin(dictionary['J']) + \
    dex_to_bin(dictionary['2']) + \
    dex_to_bin(dictionary['B'])
key_bin = []
print(len(cipher_bin))
for i in range(len(cipher_bin)):
    key_bin.append(lsfr_3.output_tail())
    lsfr_3.step_n(1)
print(cipher_bin)
print(key_bin)
plain_text_bin = np.asarray(cipher_bin) ^ np.asarray(key_bin)
print(list(plain_text_bin))

45
[0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1]
[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
[1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0]


In [119]:
plain_text_result = []
for i in range(len(plain_text_bin)//5):
    plain_text_result.append(
        alpha[
            plain_text_bin[i*5+0] * (2**4) + \
            plain_text_bin[i*5+1] * (2**3) + \
            plain_text_bin[i*5+2] * (2**2) + \
            plain_text_bin[i*5+3] * (2**1) + \
            plain_text_bin[i*5+4] * (2**0)
        ]
    )
print(plain_text_result)


['W', 'Q', '1', 'E', '1', 'U', 'U', 'D', 'O']


解密信息为WQ1E1UUDO

执行的是部分明文攻击